In [1]:
import utils

c:\Users\msalm\Desktop\BookWise-AI\utils.py:14: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(model=local_model)
c:\Users\msalm\Desktop\BookWise-AI\utils.py:20: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")  # Use SentenceTransformers for embeddings
c:\Users\msalm\anac

In [2]:
import gradio as gr

In [3]:
from utils import upload_book,chat_with_book

In [4]:
# Create Gradio interface
with gr.Blocks() as interface:
    gr.Markdown("# 📚 Book Question Answering System")
    gr.Markdown("1. **Upload a book (PDF) to process.**\n2. **Ask any question about the book and get relevant answers with references.**")
    
    # Upload Section
    with gr.Row():
        file_input = gr.File(label="Upload Book (PDF)", file_types=[".pdf"])
        upload_status = gr.Textbox(label="Upload Status", interactive=False)
    file_input.change(fn=upload_book, inputs=file_input, outputs=upload_status)

    # Question Section
    gr.Markdown("### Ask a Question About the Book")
    with gr.Row():
        question_input = gr.Textbox(label="Type your question here...", placeholder="Type your question here?")
    with gr.Row():
        answer_output = gr.Textbox(label="Answer", interactive=False)
        references_output = gr.Textbox(label="References", interactive=False)
    question_input.submit(fn=chat_with_book, inputs=question_input, outputs=[answer_output, references_output])

# Launch the interface
interface.launch(share=True)

Running on local URL:  http://127.0.0.1:7860

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


Extracted text for Page 1
Chunk created for Page Page 1: aladdin rascal Data Scientist Engineer rassaaala@g...
Chunk created for Page Page 1: com +216 21955635 rides tunisia aladdin rascal PRO...
Chunk created for Page Page 1: .g....
Final chunk for Page Page 1: , ROC curves and used Explainable AI to interpret ...
Added 4 chunks to ChromaDB.
Retrieved chunk from Page Page 123 with score 1.1893054246902466
Retrieved chunk from Page Page 2 with score 1.4853878328420924
Retrieved chunk from Page Page 1 with score 1.5245964689657545


c:\Users\msalm\Desktop\BookWise-AI\utils.py:220: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm([HumanMessage(content=prompt)])
